In [1]:
from keras.datasets import imdb

Using TensorFlow backend.


In [2]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10000)

17465344/17464789 [==============================] - 2s 0us/step


In [6]:
print('training examples', len(train_data), 'other thing', len(train_data[0]))

training examples 25000 other thing 218


In [15]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

1646592/1641221 [==============================] - 1s 0us/step


In [17]:
word_index['hello']

4822

In [18]:
reverse_word_index[666]

'mostly'

In [72]:
import string
def review_to_numbers(input_string):
    no_punc = input_string.translate(str.maketrans('', '', string.punctuation)).lower()
    word_list = no_punc.split()
    
    number_list = []
    for word in word_list:
        word_number = word_index.get(word,None)
        if word_number:
            number_list.append(word_number)
    print(number_list)

In [73]:
review_to_numbers('Hello! good sir???      How are you doing familicious')

[4822, 49, 2685, 86, 23, 22, 396]


In [68]:
def number_list_to_text(number_list):
    return ' '.join([reverse_word_index[word_number] for word_number in number_list])

In [74]:
number_list_to_text([4822, 49, 2685, 86, 23, 22, 396])

'hello good sir how are you doing'

In [101]:
# HOW IS THIS WORKING WITHOUT A SECOND FOR LOOP!!!!!?????

import numpy as np

def vectorize_reviews(reviews, dimension = 10000):
    results = np.zeros((len(reviews), dimension)) # number of training reviews x number of possible words
    for index, review in enumerate(reviews):
        results[index, review] = 1
        
    return results


In [106]:
x_train = vectorize_reviews(train_data)
x_test = vectorize_reviews(test_data)

y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

print('x_train')
print(x_train)
print()
print('y_train')
print(y_train)

x_train
[[0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 ...
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]]

y_train
[1. 0. 0. ... 0. 1. 0.]


In [107]:
# I don't understand how it worked, but somehow it did
word_list = []
for word_number, boolean in enumerate(x_train[0]):
    if boolean:
        word_list.append(reverse_word_index[word_number])
print(word_list)

['the', 'and', 'of', 'to', 'is', 'br', 'in', 'it', 'that', 'was', 'as', 'for', 'with', 'movie', 'but', 'film', 'not', 'you', 'have', 'he', 'one', 'at', 'an', 'they', 'so', 'from', 'her', 'or', 'out', 'some', 'what', 'more', 'when', 'very', 'she', 'story', 'their', 'had', 'than', 'get', 'will', 'other', 'him', 'most', 'then', 'any', 'after', 'watch', 'two', 'character', 'seen', 'never', 'acting', 'over', 'does', 'here', 'while', 'why', 'should', 'real', 'now', 'years', 'going', 'every', 'lot', 'want', 'enough', 'thought', "isn't", 'bit', 'script', 'anyone', "wasn't", 'shows', '1', 'seeing', 'half', 'help', 'mean', 'moments', 'wonderful', 'name', 'itself', 'several', 'becomes', 'unfortunately', 'history', 'heart', 'sometimes', 'themselves', 'except', 'serious', 'scary', 'room', 'whether', 'atmosphere', 'powerful', 'odd', 'visual', 'camp', 'nobody', 'loves', 'critics', 'lets', 'mine', 'current', 'boat', '70s', "show's", 'shadows', 'journalist', 'reaching', 'armed', 'chest', 'potentially',